# In-Stream Processing

The previous examples demonstrated how to use concurrency to download data, which is useful for working offline or where Internet speeds are limited. However, cloud resources allow for streaming data directly to a function without the overhead of writing the file to disk. Streaming data is more efficient regarding total time to process data.



## Streaming Data from Earthscope

This notebook demonstrates how to stream seismic miniseed files directly from FDSN servers and process them in parallel WITHOUT downloading first.

### Import packages

In [1]:
import os
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import io

### Function to Stream and Process a Single Station

In [2]:
def stream_and_process_single_station(station, start, end, network, location, channel,
                                     starttime_override, endtime_override, 
                                     output_dir, client, freqmin, freqmax,
                                     taper_percentage, corners, zerophase):
    """
    Stream miniseed data from FDSN server and process it in memory.
    
    This function combines downloading and processing into a single operation,
    streaming data directly from the server without saving raw files first.
    
    Processing steps:
    1. Stream waveform data from FDSN server
    2. Linear detrend - removes linear trends in the data
    3. Demean - removes the mean value (centers data around zero)
    4. Taper - applies a window to reduce edge effects
    5. Bandpass filter - keeps only frequencies within specified range
    6. Save processed data to output directory
    
    Parameters:
    -----------
    station : str
        Station code (e.g., 'ANMO')
    start : str
        Station's start date from metadata
    end : str
        Station's end date from metadata
    network : str
        Network code (e.g., 'XD')
    location : str
        Location code (e.g., '*' for all locations)
    channel : str
        Channel code (e.g., 'BH?' for all BH channels)
    starttime_override : str or None
        Override start time if provided
    endtime_override : str or None
        Override end time if provided
    output_dir : str
        Directory to save the processed file
    client : obspy.clients.fdsn.Client
        FDSN client instance for streaming data
    freqmin : float
        Minimum frequency for bandpass filter (Hz)
    freqmax : float
        Maximum frequency for bandpass filter (Hz)
    taper_percentage : float
        Percentage of trace to taper (0-0.5)
    corners : int
        Number of corners for Butterworth filter
    zerophase : bool
        If True, apply zero-phase filter
    
    Returns:
    --------
    tuple : (success: bool, station: str, output_file: str or None, error: str or None)
    """
    # Determine actual start/end times
    actual_start = starttime_override if starttime_override is not None else start
    actual_end = endtime_override if endtime_override is not None else end
    starttime = UTCDateTime(actual_start)
    endtime = UTCDateTime(actual_end)
    
    try:
        # Step 1: Stream waveform data directly from FDSN server
        # This happens in memory - no file is downloaded to disk
        st = client.get_waveforms(
            network=network,
            station=station,
            location=location,
            channel=channel,
            starttime=starttime,
            endtime=endtime
        )
        
        n_traces = len(st)
        
        # Step 2-5: Process each trace in memory
        for tr in st:
            # Apply linear detrend
            tr.detrend('linear')
            
            # Remove mean
            tr.detrend('demean')
            
            # Apply taper
            tr.taper(max_percentage=taper_percentage, type='hann')
            
            # Apply bandpass filter
            tr.filter('bandpass',
                     freqmin=freqmin,
                     freqmax=freqmax,
                     corners=corners,
                     zerophase=zerophase)
        
        # Step 6: Save processed data to output directory
        filename = f"{network}_{station}_{starttime.strftime('%Y%m%d')}_{endtime.strftime('%Y%m%d')}_processed.mseed"
        filepath = os.path.join(output_dir, filename)
        
        st.write(filepath, format='MSEED')
        
        print(f"✓ Streamed & processed: {station} ({n_traces} trace(s))")
        return (True, station, filepath, None)
        
    except Exception as e:
        error_msg = f"{type(e).__name__}: {str(e)}"
        print(f"✗ Failed: {station} - {error_msg}")
        return (False, station, None, error_msg)

> **Explainer:**
> 
> This function combines the download and processing steps into a single operation that happens entirely in memory. Instead of downloading raw files first, it streams waveform data directly from the FDSN server using the ObsPy client's get_waveforms() method.
> 
> The data is held in memory as an ObsPy Stream object, then immediately processed with the same four-step workflow (detrend, demean, taper, filter). Only the final processed data is saved to disk, which saves significant disk space and I/O time. This approach is ideal for processing large datasets where storing both raw and processed files would be prohibitive, and it's designed to work in parallel so multiple stations can be streamed and processed simultaneously.

### Main Parallel Streaming and Processing Function

In [3]:
def stream_and_process_parallel(station_rows, *, starttime=None, endtime=None,
                                output_dir="./processed_data",
                                freqmin=0.1, freqmax=10.0,
                                taper_percentage=0.05, corners=4,
                                zerophase=True, max_workers=5,
                                network="XD", location="*", channel="BH?",
                                fdsn_client="IRIS"):
    """
    Stream and process miniseed data from FDSN servers in parallel.
    
    This function orchestrates parallel streaming and processing of seismic data
    from multiple stations. Data is streamed directly from FDSN servers and
    processed in memory without intermediate file storage.
    
    Parameters:
    -----------
    station_rows : iterable
        Iterable of station objects with .station, .start, .end attributes
        (e.g., rows from a pandas DataFrame)
    starttime : str, optional
        Override start time in format 'YYYY-MM-DD' or 'YYYY-MM-DDTHH:MM:SS'
    endtime : str, optional
        Override end time in format 'YYYY-MM-DD' or 'YYYY-MM-DDTHH:MM:SS'
    output_dir : str, optional
        Directory to save processed files (default: './processed_data')
    freqmin : float, optional
        Minimum frequency for bandpass filter in Hz (default: 0.1)
    freqmax : float, optional
        Maximum frequency for bandpass filter in Hz (default: 10.0)
    taper_percentage : float, optional
        Percentage of trace to taper, 0-0.5 (default: 0.05 = 5%)
    corners : int, optional
        Number of corners for Butterworth filter (default: 4)
    zerophase : bool, optional
        If True, apply zero-phase filter (default: True)
    max_workers : int, optional
        Maximum number of parallel workers (default: 5)
    network : str, optional
        Network code (default: 'XD')
    location : str, optional
        Location code (default: '*')
    channel : str, optional
        Channel code (default: 'BH?')
    fdsn_client : str, optional
        FDSN client name (default: 'IRIS')
    
    Returns:
    --------
    dict : Dictionary containing 'successful' and 'failed' processing results
    
    Example:
    --------
    >>> import pandas as pd
    >>> stations_df = pd.DataFrame({
    ...     'station': ['ANMO', 'CCM', 'HLID'],
    ...     'start': ['2024-01-01', '2024-01-01', '2024-01-01'],
    ...     'end': ['2024-01-02', '2024-01-02', '2024-01-02']
    ... })
    >>> results = stream_and_process_parallel(
    ...     stations_df.itertuples(),
    ...     starttime='2024-01-01',
    ...     endtime='2024-01-02',
    ...     max_workers=10
    ... )
    """
    
    # Parse station data
    station_data = [(row.station, row.start, row.end) for row in station_rows]
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize FDSN client (thread-safe, can be shared across workers)
    client = Client(fdsn_client)
    
    # Print configuration
    print(f"{'='*70}")
    print(f"Stream & Process Seismic Data (In-Memory)")
    print(f"{'='*70}")
    print(f"FDSN server:      {fdsn_client}")
    print(f"Output directory: {output_dir}")
    print(f"Stations:         {len(station_data)}")
    print(f"Parallel workers: {max_workers}")
    print(f"\nData parameters:")
    print(f"  - Network:           {network}")
    print(f"  - Location:          {location}")
    print(f"  - Channel:           {channel}")
    if starttime:
        print(f"  - Start time:        {starttime}")
    if endtime:
        print(f"  - End time:          {endtime}")
    print(f"\nProcessing parameters:")
    print(f"  - Linear detrend:    enabled")
    print(f"  - Demean:            enabled")
    print(f"  - Taper:             {taper_percentage*100}% (Hann window)")
    print(f"  - Bandpass filter:   {freqmin}-{freqmax} Hz")
    print(f"  - Filter corners:    {corners}")
    print(f"  - Zero-phase:        {zerophase}")
    print(f"{'='*70}\n")
    
    # Initialize results
    results = {
        'successful': [],
        'failed': []
    }
    
    # Process stations in parallel using ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all streaming and processing tasks
        future_to_station = {
            executor.submit(
                stream_and_process_single_station,
                station, start, end, network, location, channel,
                starttime, endtime, output_dir, client,
                freqmin, freqmax, taper_percentage, corners, zerophase
            ): station
            for station, start, end in station_data
        }
        
        # Process results as they complete
        for future in as_completed(future_to_station):
            station = future_to_station[future]
            try:
                success, station_name, output_file, error = future.result()
                
                if success:
                    results['successful'].append({
                        'station': station_name,
                        'output_file': output_file
                    })
                else:
                    results['failed'].append({
                        'station': station_name,
                        'error': error
                    })
            except Exception as e:
                error_msg = f"Unexpected error: {str(e)}"
                print(f"✗ Failed: {station} - {error_msg}")
                results['failed'].append({
                    'station': station,
                    'error': error_msg
                })
    
    # Print summary
    print(f"\n{'='*70}")
    print(f"Processing Summary:")
    print(f"  ✓ Successful: {len(results['successful'])} stations")
    print(f"  ✗ Failed:     {len(results['failed'])} stations")
    print(f"{'='*70}")
    
    if results['failed']:
        print(f"\nFailed stations:")
        for item in results['failed']:
            print(f"  - {item['station']}: {item['error']}")
    
    return results

> **Explainer:**
> 
> This is the main orchestration function that manages the parallel streaming and processing workflow. Unlike traditional approaches that download files first and then process them, this function combines both operations into a single streaming pipeline.
> 
> It initializes a thread-safe FDSN client that's shared across all worker threads, then uses `ThreadPoolExecutor` to simultaneously stream and process data from multiple stations. Each worker thread independently connects to the FDSN server, streams waveform data into memory, processes it with the specified filters, and saves only the processed result - all without creating intermediate raw files.
>
> This approach offers several advantages:
> 1. saves disk space by not storing raw data,
> 2. reduces I/O overhead,
> 3. allows processing to begin immediately as data arrives, and
> 4.  scales efficiently for large datasets.
>
> The parallel execution means that if you're processing 20 stations with 5 workers, you'll get approximately 5x speedup compared to sequential processing, with the added benefit of reduced storage requirements.


### Usage: Basic streaming and processing

In [4]:
import pandas as pd

# Create a list of stations to process
stations_df = pd.DataFrame({
    'station': ['ANMO', 'CCM', 'HLID', 'SRU'],
    'start': ['2024-01-01', '2024-01-01', '2024-01-01', '2024-01-01'],
    'end': ['2024-01-02', '2024-01-02', '2024-01-02', '2024-01-02']
})

# Stream and process with default parameters
results = stream_and_process_parallel(
    stations_df.itertuples(),
    starttime='2024-01-01T00:00:00',
    endtime='2024-01-01T12:00:00',  # Process 12 hours of data
    output_dir='./processed_data',
    max_workers=10  # Process 10 stations simultaneously
)

Stream & Process Seismic Data (In-Memory)
FDSN server:      IRIS
Output directory: ./processed_data
Stations:         4
Parallel workers: 10

Data parameters:
  - Network:           XD
  - Location:          *
  - Channel:           BH?
  - Start time:        2024-01-01T00:00:00
  - End time:          2024-01-01T12:00:00

Processing parameters:
  - Linear detrend:    enabled
  - Demean:            enabled
  - Taper:             5.0% (Hann window)
  - Bandpass filter:   0.1-10.0 Hz
  - Filter corners:    4
  - Zero-phase:        True

✗ Failed: SRU - FDSNNoDataException: No data available for request.
HTTP Status code: 204
Detailed response of server:


✗ Failed: CCM - FDSNNoDataException: No data available for request.
HTTP Status code: 204
Detailed response of server:


✗ Failed: ANMO - FDSNNoDataException: No data available for request.
HTTP Status code: 204
Detailed response of server:


✗ Failed: HLID - FDSNNoDataException: No data available for request.
HTTP Status code: 204
Detai

> **Explainer:**
>
> This shows the simplest usage pattern where you define a list of stations (here using a pandas DataFrame) and call the main function with time boundaries. The function will stream data directly from the IRIS FDSN server for all stations in parallel, process each in memory, and save only the processed results.
>
> With 10 parallel workers, this can process 10 stations simultaneously, providing significant speedup for large station lists. The 12-hour time window is ideal for event-based studies where you want to capture a specific seismic event without downloading days of continuous data.

### Advanced Example with Custom Parameters

High-frequency processing for local events.

In [ ]:
# Process with parameters suitable for local/crustal events
results = stream_and_process_parallel(
    stations_df.itertuples(),
    starttime='2024-01-01',
    endtime='2024-01-07',  # One week of data
    output_dir='./local_events_processed',
    freqmin=5.0,           # Higher frequency for local events
    freqmax=10.0,          # Capture higher frequencies
    taper_percentage=0.1,  # 10% taper
    corners=4,
    zerophase=True,
    max_workers=8,
    network='XD',
    channel='*HZ',         # Broadband high-gain channels
    fdsn_client='IRIS'
)

print(f"\nProcessed {len(results['successful'])} stations successfully")
print(f"Failed to process {len(results['failed'])} stations")


> **Explainer:**
> 
> This example shows how to customize processing parameters for specific applications. The frequency range (5-10 Hz) is suited for local and crustal events, which have more high-frequency content than teleseismic events. Processing a full week of data demonstrates the efficiency advantage of streaming - you avoid storing 7 days × 4 stations = 28 raw files, instead only keeping the 4 processed files.
>
> The 10% taper (instead of the default 5%) provides more aggressive edge treatment, useful when you have concerns about boundary effects in the filtered data.

### Process Results and Quality Control

Analyzing and exporting results.

In [ ]:
# Display successful processing
print("\n" + "="*70)
print("Successfully Processed Stations:")
print("="*70)
for item in results['successful']:
    print(f"Station: {item['station']}")
    print(f"  Output: {item['output_file']}")
    print()

# Check for failures and investigate
if results['failed']:
    print("\n" + "="*70)
    print("Failed Stations (Need Attention):")
    print("="*70)
    for item in results['failed']:
        print(f"Station: {item['station']}")
        print(f"  Error: {item['error']}")
        print()

# Save results to CSV for documentation
successful_df = pd.DataFrame(results['successful'])
failed_df = pd.DataFrame(results['failed'])

if not successful_df.empty:
    successful_df.to_csv('streaming_successful.csv', index=False)
    print(f"✓ Saved successful results to: streaming_successful.csv")

if not failed_df.empty:
    failed_df.to_csv('streaming_failed.csv', index=False)
    print(f"✓ Saved failed results to: streaming_failed.csv")

# Calculate success rate
total = len(results['successful']) + len(results['failed'])
success_rate = len(results['successful']) / total * 100 if total > 0 else 0
print(f"\nSuccess rate: {success_rate:.1f}% ({len(results['successful'])}/{total})")

> **Explainer:**
> 
> This cell shows best practices for quality control and documentation after processing. It displays detailed information about both successful and failed stations, exports results to CSV files for record-keeping, and calculates a success rate metric. This
is particularly important when processing large numbers of stations, as it helps identify patterns in failures (e.g., certain stations might consistently fail due to data availability issues or network problems). The CSV files can be used for subsequent analysis or to retry failed stations with different parameters.

### Streaming vs Traditional Approach

**Traditional Approach (Download then Process):**

1. Download 20 stations × 10 seconds = 200 seconds
2. Process 20 files × 5 seconds = 100 seconds

   Total: 300 seconds
   Storage: Raw files + Processed files (2x storage)

Streaming Approach (This Notebook):
   - Stream + Process: 20 stations ÷ 5 workers × 10 seconds = 40 seconds
   Total: 40 seconds (~7.5x faster!)
   Storage: Only processed files (50% storage)

**Key Advantages:**

1. *Reduced Storage*: Only processed data is saved, saving 50% disk space
2. *Faster Processing*: Download and processing happen simultaneously
3. *Better Scalability*: Easier to process hundreds or thousands of stations
4. *Simplified Workflow*: No intermediate file management needed
5. *Fresh Data*: Always gets the latest data from FDSN server

**Optimal Worker Count:**

- For network-bound operations: 5-20 workers work well
- Too many workers can overwhelm FDSN servers (be respectful!)
- Balance between speed and server load
- IRIS typically allows ~5-10 concurrent connections per user

**When to Use Streaming vs Traditional:**

Use Streaming when:
- Processing many stations with limited disk space
- Need only processed data, not raw data
- Working with cloud computing (minimize storage costs)

Use Traditional (download first) when:
- Need to preserve raw data for multiple processing runs
- Working offline or with unreliable network
- Need to experiment with different processing parameters
- Archiving data for long-term studies